In [2]:
#インポート文
import numpy as np
import pandas as pd
import os
import warnings
from google.cloud import storage

In [3]:
dir = 'takamatsu/horsepage'
# race_df = pd.read_csv("../data/csv/"+dir+"/"+"race"+".csv", sep=",")
# horse_df = pd.read_csv("../data/csv/"+dir+"/"+"horse"+".csv", sep=",")

In [4]:
key_name ="../key/storage_key.json"
bucket_name = "keiba_csv_upload_bucket"
horse_info_file_upload = 'horse_info_takamatsu.csv'
horse_info_file_name = '../data/main/horse_info.csv'

In [5]:
key_name ="../key/storage_key.json"
bucket_name = "keiba_csv_upload_bucket"
horse_race_file_upload = 'horse_race_takamatsu.csv'
horse_race_file_name = '../data/main/horse_race.csv'

In [6]:
#ファイルマージ用
horse_info_df = pd.DataFrame()
horse_race_df = pd.DataFrame()
#csvファイルのマージ
for year in range(2006, 2024):
    horse_info_tmp_df = pd.read_csv("../data/csv/"+dir+"/"+str(year)+"/"+"horse-info"+".csv", sep=",")
    horse_race_tmp_df = pd.read_csv("../data/csv/"+dir+"/"+str(year)+"/"+"horse-race"+".csv", sep=",")
    horse_info_df = pd.concat([horse_info_df, horse_info_tmp_df], axis=0)
    horse_race_df = pd.concat([horse_race_df, horse_race_tmp_df], axis=0)

# horse_info_df

In [6]:
horse_info_df.head(1)

,horse_id,bday,tame_id,owner_id,producer_id,production area,auction price,winnings,lifetime record,wined race title,inbreeding-1,inbreeding-2,father,faths father,faths mother,mother,moths father,moths mother
0,1998107227,1998年4月3日,05489,083030,700317,新冠,"1,302万円 (1999年 北海道7月セレクト市場)\n","\n\n\n\n2億9,890万円 (中央) /2,040万円 (地方)\n",52戦7勝 [7-6-5-34],200606020311,1990103624,1989102140.0,1988109108,000a001bda,000a009781,1981106025,000a000258,000a00075e


##### production area

In [27]:
horse_info_df['production area'] = horse_info_df['production area'].apply(lambda x: x.replace('\n', ''))

##### auction price

In [17]:
horse_info_df['auction price'] = horse_info_df['auction price'].apply(lambda x: x.replace('\n', ''))

##### winnings

In [19]:
horse_info_df['winnings'] = horse_info_df['winnings'].apply(lambda x: x.replace('\n', ''))

##### lifetime record

In [28]:
horse_info_df['lifetime record'] = horse_info_df['lifetime record'].apply(lambda x: x.replace('\n', ''))

In [30]:
if not os.path.exists('../data/main'):
    os.makedirs('../data/main')

In [31]:
horse_info_df.to_csv("../data/main/horse_info.csv",
               index=False
              )

In [32]:
# 保管したサービスアカウントキーを指定します
client = storage.Client.from_service_account_json(key_name)

In [33]:
# 保管したサービスアカウントキーを指定します
client = storage.Client.from_service_account_json(key_name)
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(horse_info_file_upload)
blob.upload_from_filename(horse_info_file_name)

# horse_race_df

In [11]:
horse_race_df.head(1)

,date,whereracecourse,weather,race_round,race_title,race_id,total_horse_number,frame_number,horse_number,odds,...,goal_time_dif,half_way_rank,pace,last_time,horse_weight,runner-up-horse-id,prize,horse_id,race_type,surface
0,2006-03-04,2中山3,晴,11,夕刊フジオーシャンS(G3),200606020311,16,3,5,139.5,...,-0.2,12-11,33.3-35.3,34.6,494(-2),2.002107e+09,"4,157.4",1998107227,G3,芝


##### burden_weight

In [7]:
horse_race_df['burden_weight'] = horse_race_df['horse_wight']

##### horse_weight

In [11]:
horse_weight_dif = horse_race_df["horse_weight"].str.extract('\(([-|+]?\d*)\)', expand=True)
horse_weight_dif.columns ={"horse_weight_dif"}

horse_race_df = pd.concat([horse_race_df, horse_weight_dif], axis=1)

horse_race_df['horse_weight'] = horse_race_df['horse_weight'].replace('\(([-|+]?\d*)\)', '', regex=True)

In [12]:
horse_race_df['horse_weight'] = horse_race_df['horse_weight'].replace('計不', np.nan)
horse_race_df['horse_weight'] = horse_race_df['horse_weight'].astype(float)
horse_race_df['horse_weight_dif'] = horse_race_df['horse_weight_dif'].astype(float)

In [13]:
# 計不 の horse_idを探し、馬ごとの平均値で穴埋め
no_records = horse_race_df[horse_race_df['horse_weight'].isnull()]['horse_id']
for no_record_id in no_records:
    horse_race_df.loc[(horse_race_df['horse_id'] == no_record_id)&(horse_race_df['horse_weight'].isnull()), 'horse_weight'] = horse_race_df[horse_race_df['horse_id'] == no_record_id]['horse_weight'].mean() 
    horse_race_df.loc[(horse_race_df['horse_id'] == no_record_id)&(horse_race_df['horse_weight_dif'].isnull()), 'horse_weight_dif'] = 0 

In [14]:
horse_race_df.to_csv("../data/main/horse_race.csv", 
                index=False
               )

In [15]:
# 保管したサービスアカウントキーを指定します
client = storage.Client.from_service_account_json(key_name)

In [16]:
# 保管したサービスアカウントキーを指定します
client = storage.Client.from_service_account_json(key_name)
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(horse_race_file_upload)
blob.upload_from_filename(horse_race_file_name)

In [17]:
horse_race_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7374 entries, 0 to 343
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                7374 non-null   object 
 1   whereracecourse     7374 non-null   object 
 2   weather             7374 non-null   object 
 3   race_round          7374 non-null   object 
 4   race_title          7374 non-null   object 
 5   race_id             7374 non-null   object 
 6   total_horse_number  7374 non-null   int64  
 7   frame_number        7374 non-null   object 
 8   horse_number        7374 non-null   int64  
 9   odds                7374 non-null   float64
 10  popular             7374 non-null   int64  
 11  rank                7374 non-null   int64  
 12  rider_id            7374 non-null   int64  
 13  horse_wight         7374 non-null   float64
 14  distance            7374 non-null   int64  
 15  groud_status        7374 non-null   object 
 16  goal_ti

In [16]:
horse_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 321 entries, 0 to 17
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   horse_id          321 non-null    int64 
 1   bday              321 non-null    object
 2   tame_id           321 non-null    object
 3   owner_id          321 non-null    object
 4   producer_id       319 non-null    object
 5   production area   321 non-null    object
 6   auction price     321 non-null    object
 7   winnings          321 non-null    object
 8   lifetime record   321 non-null    object
 9   wined race title  321 non-null    object
 10  inbreeding-1      314 non-null    object
 11  inbreeding-2      244 non-null    object
 12  father            321 non-null    object
 13  faths father      321 non-null    object
 14  faths mother      321 non-null    object
 15  mother            321 non-null    object
 16  moths father      321 non-null    object
 17  moths mother     